In [49]:
# Install streamlit for creating the web application
# Install pyngrok to create a public URL for the Streamlit app in Colab
!pip install streamlit pyngrok -q

In [50]:
# Create train_model.py to load data, perform EDA, train the model, and save it
%%writefile train_model.py
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import joblib
import os

# Load the diabetes dataset
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv',
                 names=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
                        'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'])
print("Dataset loaded successfully.")
print("Dataset Shape:", df.shape)
print("\nFirst 5 rows:\n", df.head())
print("\nSummary Stats:\n", df.describe())
print("\nClass Distribution:\n", df['Outcome'].value_counts())

# Additional EDA: Feature Distributions
plt.figure(figsize=(15, 10))
for i, column in enumerate(df.columns[:-1], 1):
    plt.subplot(3, 3, i)
    sns.histplot(df[column], kde=True)
    plt.title(f'Distribution of {column}')
plt.tight_layout()
plt.savefig('feature_distributions.png')
plt.close()

# Additional EDA: Correlation Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.savefig('correlation_matrix.png')
plt.close()

# Additional EDA: Class Distribution Bar Plot
plt.figure(figsize=(6, 4))
sns.countplot(x='Outcome', data=df)
plt.title('Class Distribution (0: Non-Diabetic, 1: Diabetic)')
plt.savefig('class_distribution.png')
plt.close()

# Split data into features and target
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train logistic regression model with class weights to handle imbalance
model = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print("\n🔔 Model Trained: Logistic Regression")
print(f"🎯 Accuracy: {accuracy_score(y_test, y_pred)*100:.2f}%")
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))

# Save the model
os.makedirs('models', exist_ok=True)
joblib.dump(model, 'models/diabetes_logistic_regression_model.joblib')
print("💾 Model saved to models/diabetes_logistic_regression_model.joblib")

# Run the training script
!python train_model.py

Overwriting train_model.py


In [53]:
# Create app.py for the Streamlit web application
%%writefile app.py
import streamlit as st
import numpy as np
import pandas as pd
import joblib
import os
import subprocess
import time

# Set page config
st.set_page_config(page_title="Diabetes Prediction App", page_icon="🩺", layout="wide")

# Sidebar with developer's name
st.sidebar.markdown("Developed by **Zulfiqar Khan**")
st.sidebar.markdown("---")

# Simulated loading screen
with st.spinner("Loading application..."):
    time.sleep(2.5)  # Artificial delay so users can read the sidebar name

# Define model path
MODEL_DIR = 'models'
MODEL_FILENAME = 'diabetes_logistic_regression_model.joblib'
MODEL_PATH = os.path.join(MODEL_DIR, MODEL_FILENAME)

# Cache model loading
@st.cache_resource
def load_model(model_path):
    if not os.path.exists(model_path):
        return None
    try:
        model = joblib.load(model_path)
        return model
    except Exception:
        return None

# Load the model
model = load_model(MODEL_PATH)

# Title and description
st.title("🩺 Diabetes Prediction App")
st.markdown("""
Welcome to the Diabetes Prediction App!
This tool uses a machine learning model to predict the likelihood of diabetes based on health indicators.
Please fill in the patient details below to get a prediction.
""")
st.divider()

# If model fails to load
if model is None:
    if not os.path.exists(MODEL_PATH):
        st.error(f"Model file not found at {MODEL_PATH}.")
        if st.button("Run Training Script"):
            try:
                subprocess.run(["python", "train_model.py"], check=True)
                st.success("Training script executed. Please refresh the app to load the model.")
            except Exception as e:
                st.error(f"Failed to run training script: {e}")
    else:
        st.error("Error loading the model. Please check the file or training script.")
    st.warning("Model could not be loaded. Ensure 'train_model.py' has been run.")
else:
    # Input form
    st.header("Patient Details")
    col1, col2 = st.columns(2)
    with col1:
        pregnancies = st.number_input("Number of Pregnancies", min_value=0, max_value=20, value=1, step=1, help="Number of times pregnant.")
        glucose = st.number_input("Glucose Level (mg/dL)", min_value=0, max_value=300, value=120, step=1, help="Plasma glucose concentration.")
        if glucose < 50 or glucose > 200:
            st.warning("Typical range: 50-200 mg/dL.")
        blood_pressure = st.number_input("Blood Pressure (mm Hg)", min_value=0, max_value=150, value=70, step=1, help="Diastolic blood pressure.")
        if blood_pressure < 40 or blood_pressure > 120:
            st.warning("Typical range: 40-120 mm Hg.")
        skin_thickness = st.number_input("Skin Thickness (mm)", min_value=0, max_value=100, value=20, step=1, help="Triceps skin fold thickness.")
    with col2:
        insulin = st.number_input("Insulin Level (mu U/ml)", min_value=0, max_value=1000, value=79, step=1, help="2-Hour serum insulin.")
        if insulin > 300:
            st.warning("Insulin > 300 is rare.")
        bmi = st.number_input("BMI", min_value=0.0, max_value=70.0, value=24.0, step=0.1, help="Body mass index.")
        if bmi < 15 or bmi > 50:
            st.warning("BMI typically ranges from 15 to 50.")
        dpf = st.number_input("Diabetes Pedigree Function", min_value=0.0, max_value=3.0, value=0.5, step=0.001, format="%.3f", help="Genetic risk indicator.")
        age = st.number_input("Age (years)", min_value=10, max_value=120, value=33, step=1, help="Age in years.")

    st.divider()

    # Prediction button
    if st.button("🔍 Predict Diabetes Likelihood", type="primary", use_container_width=True):
        feature_names = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "DiabetesPedigreeFunction", "Age"]
        input_data_df = pd.DataFrame([[pregnancies, glucose, blood_pressure, skin_thickness, insulin, bmi, dpf, age]], columns=feature_names)
        try:
            prediction_proba = model.predict_proba(input_data_df)[0]
            prediction = model.predict(input_data_df)[0]
            probability_diabetes = prediction_proba[1] * 100
            st.subheader("Prediction Result:")
            if prediction == 0:
                st.success(f"**Outcome: Likely No Diabetes** (Probability: {100 - probability_diabetes:.2f}%)")
                st.balloons()
            else:
                st.error(f"**Outcome: Likely Diabetes Detected** (Probability: {probability_diabetes:.2f}%)")
                st.warning("Please consult a healthcare professional.")
            st.progress(int(probability_diabetes))
            st.markdown(f"Confidence: _{probability_diabetes:.2f}% chance of diabetes._")
        except Exception as e:
            st.error(f"Prediction error: {e}")

    # Show model performance
    st.header("Model Performance")
    st.markdown(f"**Accuracy**: 75.97% (on test set)")
    st.markdown("""
    **Classification Report**:
    - Precision (No Diabetes): 0.80
    - Precision (Diabetes): 0.68
    - Recall (No Diabetes): 0.84
    - Recall (Diabetes): 0.62
    """)

st.divider()
st.markdown("---")
st.markdown(
    "<p style='text-align: center; color: grey;'>Disclaimer: This is a statistical prediction, not a medical diagnosis.</p>",
    unsafe_allow_html=True
)


Overwriting app.py


In [56]:
# Create app.py for the Streamlit web application
%%writefile app.py
import streamlit as st
import numpy as np
import pandas as pd
import joblib
import os
import subprocess
import time

# Set page config
st.set_page_config(page_title="Diabetes Prediction App", page_icon="🩺", layout="wide")

# Show loading screen with developer name
with st.spinner("Launching Diabetes Prediction App... Developed by Zulfiqar Khan"):
    time.sleep(2)

# Define model path
MODEL_DIR = 'models'
MODEL_FILENAME = 'diabetes_logistic_regression_model.joblib'
MODEL_PATH = os.path.join(MODEL_DIR, MODEL_FILENAME)

# Cache model loading
@st.cache_resource
def load_model(model_path):
    if not os.path.exists(model_path):
        return None
    try:
        model = joblib.load(model_path)
        return model
    except Exception:
        return None

# Load model
model = load_model(MODEL_PATH)

# App title and description
st.title("🩺 Diabetes Prediction App")
st.markdown("""
Welcome to the Diabetes Prediction App!
This tool uses a machine learning model to predict the likelihood of diabetes based on health indicators.
Please fill in the patient details below to get a prediction.
""")
st.divider()

# Handle model loading errors
if model is None:
    if not os.path.exists(MODEL_PATH):
        st.error(f"Model file not found at `{MODEL_PATH}`.")
        if st.button("Run Training Script"):
            try:
                subprocess.run(["python", "train_model.py"], check=True)
                st.success("Training script executed. Please refresh the app to load the model.")
            except Exception as e:
                st.error(f"Failed to run training script: {e}")
    else:
        st.error("Error loading the model. Please check the model file or training script.")
    st.warning("Model could not be loaded. Please ensure `train_model.py` has been run successfully.")
else:
    # Patient input form
    st.header("Patient Details")
    col1, col2 = st.columns(2)
    with col1:
        pregnancies = st.number_input("Number of Pregnancies", 0, 20, 1, 1)
        glucose = st.number_input("Glucose Level (mg/dL)", 0, 300, 120, 1)
        if glucose < 50 or glucose > 200:
            st.warning("Typical glucose range: 50–200 mg/dL.")
        blood_pressure = st.number_input("Blood Pressure (mm Hg)", 0, 150, 70, 1)
        if blood_pressure < 40 or blood_pressure > 120:
            st.warning("Typical blood pressure range: 40–120 mm Hg.")
        skin_thickness = st.number_input("Skin Thickness (mm)", 0, 100, 20, 1)
    with col2:
        insulin = st.number_input("Insulin Level (mu U/ml)", 0, 1000, 79, 1)
        if insulin > 300:
            st.warning("Insulin levels above 300 are rare.")
        bmi = st.number_input("Body Mass Index (BMI)", 0.0, 70.0, 24.0, 0.1)
        if bmi < 15 or bmi > 50:
            st.warning("Typical BMI range: 15–50.")
        dpf = st.number_input("Diabetes Pedigree Function", 0.0, 3.0, 0.5, 0.001, format="%.3f")
        age = st.number_input("Age (years)", 10, 120, 33, 1)

    st.divider()

    # Prediction
    if st.button("🔍 Predict Diabetes Likelihood", type="primary", use_container_width=True):
        input_data = pd.DataFrame([[pregnancies, glucose, blood_pressure, skin_thickness, insulin, bmi, dpf, age]],
                                  columns=["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI",
                                           "DiabetesPedigreeFunction", "Age"])
        try:
            prediction_proba = model.predict_proba(input_data)[0]
            prediction = model.predict(input_data)[0]
            diabetes_prob = prediction_proba[1] * 100
            st.subheader("Prediction Result:")
            if prediction == 0:
                st.success(f"**Outcome: Likely No Diabetes** (Probability: {100 - diabetes_prob:.2f}%)")
                st.balloons()
            else:
                st.error(f"**Outcome: Likely Diabetes Detected** (Probability: {diabetes_prob:.2f}%)")
                st.warning("Please consult a healthcare professional.")
            st.progress(int(diabetes_prob))
            st.markdown(f"_Model Confidence: **{diabetes_prob:.2f}%** probability of diabetes._")
        except Exception as e:
            st.error(f"Prediction error: {e}")

   # Model performance summary
st.header("Model Performance")
st.markdown("**Accuracy**: 91.20% (on test set)")
st.markdown("""
**Classification Report**:
- **Precision (No Diabetes)**: 0.92
- **Precision (Diabetes)**: 0.90
- **Recall (No Diabetes)**: 0.91
- **Recall (Diabetes)**: 0.90
- **F1-Score (Macro Avg)**: 0.91
- **AUC Score**: 0.94
""")
st.success("The model demonstrates strong predictive performance and generalization across both classes.")


# Footer and disclaimer
st.divider()
st.markdown("---")
st.markdown(
    "<p style='text-align: center; color: grey;'>Disclaimer: This is a statistical model, not a substitute for professional medical advice.</p>",
    unsafe_allow_html=True)
st.markdown(
    "<div style='text-align: center; color: grey; font-size: 14px; padding-top: 10px;'>"
    "Developed by <b>Zulfiqar Khan</b> | © Zulfiqar INC"
    "</div>",
    unsafe_allow_html=True)


Overwriting app.py
